In [69]:
import bs4
import requests
from time import sleep
import pandas as pd
import numpy as np
import pickle

In [85]:
data_temperature = []
data_temperature_wind = []
data_wind = []

start_time = '2015-08-01 00:00:00'
end_time = '2016-07-02 00:00:00'
p = pd.date_range(start=start_time, end=end_time, freq='D')
weather_time_index = pd.date_range(start=start_time, end=end_time, freq='H')

Weather parsing from weather https://www.wunderground.com/history/

In [86]:
%%time
date_weather = pd.DataFrame(columns=['temperature', 'wind', 'time'], index=weather_time_index)

# в некоторые часы наблюдается много данных (по минутам), будем брать первые, остальные пропускать
# надеюсь, что данные есть по всем нужным дням-часам(хотя бы без больших пропусков)
# в случае отсутсвия берутся последние
previous_hour = -1

index_counter = 0
for date in p: 
    if(not date):
        break
        
     # задержки для неравномерности запросов 
    sleep(0.1*np.random.randint(1, 10))
    req = requests.get('https://www.wunderground.com/history/airport/KNYC/{}/{}/{}/'\
          'DailyHistory.html?req_city=New+York&req_state=NY&req_statename=New+York&reqdb.'\
          'zip=10001&reqdb.magic=11&reqdb.wmo=99999&MR=1'.format(date.year, date.month, date.day))
    
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    p1 = parser.findAll('tr', attrs={'class': "no-metars"})
    print (date)
    for res in p1:
        # hour hh:51
        child = res.findChild('td')
        if(child):
            time = child.text
            hour = int(time[:time.find(':')])
            
            # приводим к интервалу 0-23 часа
            if(time.find('PM') != -1 and hour != 12):
                hour += 12
            if(time.find('AM') != -1 and hour == 12):
                hour = 0
                
            # пропускаем много данных за час    
            if(previous_hour == hour):
                continue
            
            previous_hour = hour
            
        
        #temperature
        child = res.findChild('span', attrs={'class': 'wx-value'})
        if (child):
            temperature = child.text
            child = child.findNext('span', attrs={'class': 'wx-value'})
            if(child):
                wind = child.text
            else:
                print("error: no wind data for: ", hour, date)
        else:
            print("error: no temperature data for: ", hour, date)

        date_weather['temperature'][index_counter] = temperature
        date_weather['wind'][index_counter] = wind
        date_weather['time'][index_counter] = time
        index_counter += 1 
        

2015-08-01 00:00:00
2015-08-02 00:00:00
2015-08-03 00:00:00
2015-08-04 00:00:00
2015-08-05 00:00:00
2015-08-06 00:00:00
2015-08-07 00:00:00
2015-08-08 00:00:00
2015-08-09 00:00:00
2015-08-10 00:00:00
2015-08-11 00:00:00
2015-08-12 00:00:00
2015-08-13 00:00:00
2015-08-14 00:00:00
2015-08-15 00:00:00
2015-08-16 00:00:00
2015-08-17 00:00:00
2015-08-18 00:00:00
2015-08-19 00:00:00
2015-08-20 00:00:00
2015-08-21 00:00:00
2015-08-22 00:00:00
2015-08-23 00:00:00
2015-08-24 00:00:00
2015-08-25 00:00:00
2015-08-26 00:00:00
2015-08-27 00:00:00
2015-08-28 00:00:00
2015-08-29 00:00:00
2015-08-30 00:00:00
2015-08-31 00:00:00
2015-09-01 00:00:00
2015-09-02 00:00:00
2015-09-03 00:00:00
2015-09-04 00:00:00
2015-09-05 00:00:00
2015-09-06 00:00:00
2015-09-07 00:00:00
2015-09-08 00:00:00
2015-09-09 00:00:00
2015-09-10 00:00:00
2015-09-11 00:00:00
2015-09-12 00:00:00
2015-09-13 00:00:00
2015-09-14 00:00:00
2015-09-15 00:00:00
2015-09-16 00:00:00
2015-09-17 00:00:00
2015-09-18 00:00:00
2015-09-19 00:00:00


In [88]:
date_weather.to_csv('weather.csv')